# Access GBIF data through R

## Table of contents
1. [Setting up](#setup)
2. [Loading R packages](#packages)
3. [Getting species data from GBIF](#get-data)
4. [Cleaning species occurrence data](#clean-data)
5. [Plot species occurrence data on a map](#plot-data)

## Setting up <a name="setup"></a>

If you opened this notebook from the GitHub *ecocloud* repository, please save it first to your own workspace to ensure that any of your changes will be saved. To create a new folder and save this notebook to your workspace follow these steps:
1. Click on the 'folder' icon in the menubar on the left (top icon).
2. Double click on the workspace folder.
3. Click on the 'new folder' icon in the menubar on the top. This will create a new Untitled Folder.
4. Right click on the Untitled Folder, click Rename, and name it appropriately.
5. Double click on the folder name to move into the folder.
6. Go to 'File' > 'Save Notebook'.
7. It is really important that when you save your notebook, you save it in the following format: "workspace/Folder Name/my-notebook.ipynb". Without the "workspace/" in the name the notebook will not be saved in the workspace for future use. 

**Working with a Jupyter Notebook**

You can execute cells of code in a notebook by clicking on the cell and then press "Shift+Enter" on your keyboard, or click the "Play" icon in the menubar. This will execute the current cell, show any output, and jump to the next cell below. During execution of a cell you will see a * next to the cell.

Set your working directory to the folder you just created, by executing the code in the cell below. NB. Change the text *biodiversity_next* to the name of your folder.

In [ ]:
setwd("/home/jovyan/workspace/biodiversity_next")
getwd()

## Loading R packages <a name="packages"></a>

*ecocloud* has pre-installed many R packages that are often used by ecology researchers, but you can install packages yourself if needed (use install.packages command). The cell below loads the libraries needed to execute the script in this notebook.

In [ ]:
library(dismo)
library(rgdal)
library(sp)
library(maptools)

## Getting species data from GBIF <a name="get-data"></a>

The [Global Biodiversity Information Facility (GBIF)](https://www.gbif.org/) is an international network that provides open access to data about all types of life on Earth.

To access GBIF occurrence data for a particular species, we are using the 'gbif' function in the 'dismo' R package. Ensure that you are familiar with the [GBIF data use agreement](https://www.gbif.org/terms) before using this function.

To use the function, you need to put the genus and species name separately between brackets: 

gbif('genus', 'species')

Other arguments can be found in the [documentation](https://www.rdocumentation.org/packages/dismo/versions/1.1-4/topics/gbif). Here we include:
- geo = TRUE : only download records that have a georeference (longitude and latitude values). 
- removeZeros = TRUE : removes all records that have a latitude OR longitude of zero.
- download = FALSE/TRUE : use FALSE if you want to check first how many records there are before starting download.

For example, if you want to download occurrence records of the American pika (*Ochotona princeps*), you would use:

`pika <- gbif('Ochotona', 'princeps', geo=TRUE, removeZeros=TRUE, download=FALSE)`

`pika` - to show the number of records

Try this yourself with a species of interest. First check number of records with `download = FALSE`, for the sake of time please select a species with less than ~20,000 records, then change to `download = TRUE` to download the records into your working environment.

Have a look at the data by running the following code in which you change ... to the name of your dataframe:

`head(...)` shows the first six rows of the dataset

`str(...)` shows the structure of the dataframe listing all the columns

To keep a copy of the full dataset, save the data as a csv file adapting this code (df = name of dataframe, change species-name to your species):

`write.csv(df, file="species-name.csv", row.names=TRUE)`

Load the data as a dataframe with a generic name (easy for future use with a different species) with this code (df = name of dataframe):

`spp.data <- df`

## Cleaning species occurrence data <a name="clean-data"></a>
While it is useful to keep the entire file for provenance, we don't need to keep all columns for further analysis.

*Think about what information here is crucial and what is unnecessary?*

We want to create a new dataframe with only a selection of columns. For this exercise we would like to keep the ID number, species name, latitude, longitude, year and coordinate uncertainty. 

You could do this using the following code:

`records <- data.frame(
  idNum=c(spp.data$gbifID),
  species=c(spp.data$acceptedScientificName),
  ...`

In this code spp.data refers to the dataframe, the name behind $ refers to the actual column within this dataframe.


Finish this code yourself by adding: 

`longitude = , 
latitude = , 
year = , 
coordUncer =
)`

Check the resulting dataframe by executing the following code:

In [ ]:
head(records)
summary(records)

**Remove occurrence records with missing data**

Although in the gbif function we set removeZeros=TRUE, their might still be still records with longitude/latitude = NA in the dataset, so you can remove these manually by executing the code cell below. The first line removes records with longitude or latitude = NA, 'nrow' shows you the number of records left, 'head' gives you again the first six lines of the dataset.

In [ ]:
records <- records[-which(is.na(records$longitude) | is.na(records$latitude)), ]
nrow(records)
head(records)

**Check the coordinate uncertainty of the occurrence data**

You might want to include data that have a certain level of coordinate certainty. Use the following code to check the minimum and maximum values for coordinate uncertainty in your dataset.

`min(records$coordUncer, na.rm=TRUE)`

`max(records$coordUncer, na.rm=TRUE)`

If your datasets includes records with large coordinate uncertainty you might want to remove those. You can remove all records with a coordinate uncertainty that is larger than 10km by using the following code:

`records.selection <- records[records$coordUncer<=10000, ]`

Feel free to change the maximum uncertainty to what suits your needs and run the code. Use the code from above to double check the minumum and maximum coordinate uncertainty in the new dataframe (records.selection), and check the number of records left using the command 'nrows'.

A good way to look at data like this is to plot it in a histogram, which you can do by executing the cell below:

In [ ]:
hist(records.selection$coordUncer,
     xlab='Coordinate Uncertainty',
     ylab='Number of records',
     main='Coordinate uncertainty (m)'
)

**Check the year range of the occurrence data**

Depending on what you want to use this data for you might want to exclude records from particular year ranges. For example, if you use the species data in a Species Distribution Model using current climate data, you might want to consider only including occurrence records from the years matching the climate data.

Using the data that you just filtered for the selected coordinate certainty values, you can use the code below to plot a histogram of the years of the data, and list the earliest and latest year:

In [ ]:
hist(records.selection$year,
     xlab='Collection year',
     ylab='Number of records',
     main='Collection year (all years)'
)

min(records.selection$year, na.rm=TRUE)
max(records.selection$year, na.rm=TRUE)

Make a selection of records within a given year range adapting the code below:

`records.selection <- records.selection[records.selection$year >=1970 & records.selection$year <=2000, ]`

You can then redraw the histogram copying the code from above.

## Plot species occurrence data on a map <a name="plot-data"></a>

Now you have made a selection, let's plot the occurrences on a map. Depending on where in the world your species is found, you might need to adjust the 'xlim' and 'ylim' values to get the right area in the frame. You can also delete these arguments to show the entire worldmap. Have a play around to see what works best.

In [ ]:
data(wrld_simpl) #load world country polygons from maptools package
plot(wrld_simpl, xlim=c(-130,-60), ylim=c(10,60), axes=TRUE, col="light yellow")
box() #add box around map
points(records.selection$longitude, records.selection$latitude, col="red", pch=20, cex=0.75)